# Plant Disease Detection Group Project

## Machine Learning - UFCFAS-15-2

### GroupMembers
#### Ahmed Elsaman - 21072727
#### Hatim Shaherawla - 21054059
#### Tommy Diclaudio - 21035734

Importing the necessary libraries

In [ ]:
import pandas as pd                             # Importing pandas library for data manipulation
import pathlib                                   # Importing pathlib library for file path manipulation
import tensorflow as tf                          # Importing TensorFlow library for deep learning
import matplotlib.pyplot as plt                  # Importing matplotlib library for data visualization
import numpy as np                               # Importing NumPy library for numerical computation
import os                                        # Importing os library for operating system related tasks
import PIL                                       # Importing Python Imaging Library for image processing
import glob                                      # Importing glob library for file path pattern matching
from tensorflow import keras                     # Importing keras module from TensorFlow for building deep learning models
from keras import layers                         # Importing layers module from keras for building deep learning models
from keras.models import Sequential              # Importing Sequential model from keras for building deep learning models
from keras.utils import image_dataset_from_directory  # Importing image_dataset_from_directory function from keras for loading images
from keras.preprocessing.image import ImageDataGenerator  # Importing ImageDataGenerator class from keras for data augmentation
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, BatchNormalization  # Importing different types of layers from keras for building deep learning models
from keras.losses import SparseCategoricalCrossentropy  # Importing SparseCategoricalCrossentropy loss function from keras
from keras.regularizers import l2                 # Importing l2 regularizer from keras for regularization
import os                                        # Importing os library for operating system related tasks

%matplotlib inline                               # Magic command for displaying plots inline in Jupyter Notebook
from glob import glob                            # Importing glob library for file path pattern matching
import seaborn as sns                            # Importing seaborn library for data visualization
from PIL import Image                            # Importing Python Imaging Library for image processing
np.random.seed(11)                               # Setting seed for reproducibility
from sklearn.preprocessing import StandardScaler # Importing StandardScaler class from scikit-learn for scaling data
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV # Importing different modules from scikit-learn for model evaluation and tuning
from sklearn.metrics import accuracy_score       # Importing accuracy_score function from scikit-learn for computing model accuracy
import itertools                                 # Importing itertools library for various combinatorial functions

import keras                                     # Importing keras library for deep learning
from keras.utils.np_utils import to_categorical # Importing to_categorical function from keras for one-hot encoding of target variable
from keras.models import Sequential, Model       # Importing different modules from keras for building deep learning models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D # Importing different types of layers from keras for building deep learning models
from keras import backend as K                   # Importing backend module from keras for backend-specific functions
from keras.layers import BatchNormalization     # Importing BatchNormalization layer from keras for normalization
from keras.optimizers import Adam, RMSprop       # Importing Adam and RMSprop optimizers from keras for model optimization
from keras.preprocessing.image import ImageDataGenerator  # Importing ImageDataGenerator class from keras for data augmentation
from keras.callbacks import ReduceLROnPlateau    # Importing ReduceLROnPlateau class from keras for learning rate scheduling
from keras.wrappers.scikit_learn import KerasClassifier  # Importing KerasClassifier wrapper from scikit-learn for using keras models with scikit-learn API
from keras.applications.inception_v3 import InceptionV3   # Importing InceptionV3 pre-trained model from keras
from keras import backend as K
import random  # for generating random numbers
import urllib.request  # for downloading images from URLs
import matplotlib.image as mpimg  # for loading images as arrays

from skimage.filters import rank, threshold_otsu  # for applying image filters
from skimage import io  # for reading and displaying images
from skimage.color import rgb2gray  # for converting images to grayscale
from sklearn.cluster import KMeans  # for applying k-means clustering algorithm
from skimage.morphology import closing, square, disk  # for applying morphological operations on images



Defining the path to the original dataset


In [ ]:


data_dir = pathlib.Path("archive/plantvillage dataset/color")



Gettng list of directories in data directory and print it, then print the amount of directories found.

In [ ]:

dataset_path_train = os.listdir(data_dir)   
print (dataset_path_train)
print("Amount of classes found: ", len(dataset_path_train))

This sets the batch size for the image data generator and the image height and width for the images in the dataset.

In [ ]:

batch_size = 32
img_height = 224
img_width = 224

### Transfer Learning with Fine-Tuning using VGG16 for Multi-Class Image Classification

This code uses the Keras library to prepare the data for the image classification task by applying data augmentation techniques. It then loads a pre-trained VGG16 model and freezes all its layers except for the last fully connected layers. A new fully connected layer is added and then the model is compiled. Finally, the last 10 layers of the model are unfrozen for fine-tuning.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import SGD

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

# Transfer learning
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(38, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Fine-tuning
for layer in model.layers[-10:]:
    layer.trainable = True

In [ ]:
# Hyperparameter tuning
optimizer = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


Training the model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator)

Evaluating the model

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Get the predicted labels for the test set
test_pred = model.predict(test_generator)
test_pred_labels = np.argmax(test_pred, axis=1)

# Get the true labels for the test set
test_true_labels = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(test_true_labels, test_pred_labels)
print(cm)


This code sample performs image prediction using a trained TensorFlow model, and displays the predicted and actual class labels along with their respective confidence levels.

In [ ]:
from tensorflow import expand_dims

labels = list(test_generator.class_indices.keys())

def predict(model, img):
    
    img_array = img.numpy()
    img_array = expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = labels[np.argmax(predictions[0])]
    confidence = round(np.max(predictions[0]) * 100, 2)
    
    return predicted_class, confidence

plt.figure(figsize=(15, 15))

for images, lbs in val_ds.take(1):
    for i in range(12):
        
        plt.subplot(4, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint32"))
        
        predicted_class, confidence = predict(model, images[i])
        actual_class = labels[lbs[i]] 
        
        title = f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%"
        if actual_class == predicted_class:
            plt.title(title, color="green")
        else:
            plt.title(title, color="red")
        
        plt.axis("off")
        
        plt.tight_layout()


Saving the model

In [ ]:
model.save("Model.h5")